这里主要是应用pytorch 做深度学习编程。
假设你了解NLP的主要问题：part-of-speech tagging, 语言模型等。并且假设你熟悉神经网络，包括 基本的backpropagation on feed-forward neural networks.

In [53]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

## 1 介绍 torch的张量库
所有深度学习都是基于张量来计算的，张量可以理解为矩阵（维度可以超过2维）。

首先，我们看下用张量我们可以做什么。

### 创建张量
张量可以通过list 用 torch.Tensor()来创建。默认是Float数值。
如果要创建Int数值类型的 tensor， 可以用 torch.LongTensor(). Float/Long是常用的参数类型。

In [6]:
V_data = [1, 2, 3]
V = torch.Tensor(V_data)
print(V)

M_data = [[1,2],[3,4]]
M = torch.Tensor(M_data)
print(M)

T_data = [[[1,2],[3,4]],[[5,6],[7,8]]]
T = torch.Tensor(T_data)
print(T)

tensor([1., 2., 3.])
tensor([[1., 2.],
        [3., 4.]])
tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])


In [7]:
print(V[0], M[0], T[0])  ## index 分别获得 1数值，1个向量，2个矩阵

tensor(1.) tensor([1., 2.]) tensor([[1., 2.],
        [3., 4.]])


In [9]:
x = torch.randn((3,4,5))  ## 通过 torch.randn(n,x,y) 获得n维 x*y的张量
print(x) 

tensor([[[ 0.4100,  0.4085,  0.2579,  1.0950, -0.5065],
         [ 0.0998, -0.6540,  0.7317, -1.4567,  1.6089],
         [ 0.0938, -1.2597,  0.2546, -0.5020, -1.0412],
         [ 0.7323,  1.3075, -1.1628,  0.1196, -0.1631]],

        [[ 0.6614,  1.1899,  0.8165, -0.9135, -0.3538],
         [ 0.7639, -0.5890, -0.7636,  1.3352,  0.6043],
         [-0.1034, -0.1512,  1.2466,  0.5057,  0.9505],
         [ 1.2966,  0.8738, -0.5603,  1.2858,  0.8168]],

        [[-1.4648, -1.2629,  1.1220,  1.5663, -1.0371],
         [-1.0669, -0.2085, -0.2155,  0.2705,  0.5597],
         [-0.3184,  1.5117, -1.5208,  0.9196, -0.5484],
         [-0.3472, -0.7474, -0.9234,  0.5734, -0.1093]]])


### 张量操作

In [10]:
x = torch.Tensor([1,2,3,4])
y = torch.Tensor([4,5,0,9])
z = x+y
print(z) ## 浮点型 float

tensor([ 5.,  7.,  3., 13.])


In [11]:
x = torch.LongTensor([1,2,3,4])
y = torch.LongTensor([4,5,0,9])
z = x+y
print(z) ## 长整型

tensor([ 5,  7,  3, 13])


In [13]:
torch.Tensor([1,2,3]).dtype

torch.float32

In [14]:
##torch.set_default_dtype(torch.float64)
torch.get_default_dtype()

torch.float32

In [16]:
a = torch.randn(1,2,3,4,5)
torch.numel(a) ## 返回乘积

120

张量的操作可以参考 [the documentation](http://pytorch.org/docs/torch.html) 
我们待会儿要用到的操作是 concatenation（连接）

In [22]:
x1 = torch.randn(2,5)
y1 = torch.randn(3,5)
z1 = torch.cat([x1, y1])  ## 默认 是基于第一维连接。 first  axis, 所以 z1 = (2+3, 5)
print(x1)
print(y1)
print(z1) 

x2 = torch.randn(2,3)
y2 = torch.randn(2,5)
z2 = torch.cat([x2, y2], 1) ## 设置从第二维度连接  z2 = (2, 3+5)
print(x2)
print(y2)
print(z2)

tensor([[-0.8678,  0.5870, -0.1618, -1.3426,  0.8099],
        [ 1.0417,  0.4967,  1.7153, -1.1099,  0.3573]])
tensor([[-0.3369, -0.1951,  1.6927, -0.9122, -0.1971],
        [ 0.6831,  0.6439, -1.5348,  0.1530, -0.3657],
        [ 0.0245, -0.0813,  1.5079,  1.1710,  0.6938]])
tensor([[-0.8678,  0.5870, -0.1618, -1.3426,  0.8099],
        [ 1.0417,  0.4967,  1.7153, -1.1099,  0.3573],
        [-0.3369, -0.1951,  1.6927, -0.9122, -0.1971],
        [ 0.6831,  0.6439, -1.5348,  0.1530, -0.3657],
        [ 0.0245, -0.0813,  1.5079,  1.1710,  0.6938]])
tensor([[0.8491, 0.2730, 0.3795],
        [0.0478, 1.3501, 0.0977]])
tensor([[-1.4379,  1.8068, -1.2562,  1.6601, -0.9056],
        [-0.5866,  1.0355,  0.6981,  1.1381, -0.4953]])
tensor([[ 0.8491,  0.2730,  0.3795, -1.4379,  1.8068, -1.2562,  1.6601, -0.9056],
        [ 0.0478,  1.3501,  0.0977, -0.5866,  1.0355,  0.6981,  1.1381, -0.4953]])


### Reshaping Tensor
通过 .view() 可以进行张量的形状转换。
Use the .view() method to reshape a tensor. This method receives heavy use, because many neural network components expect their inputs to have a certain shape. Often you will need to reshape before passing your data to the component.

In [23]:
x = torch.randn(2,3,4)
print(x)
print(x.view(2,12))
print(x.view(2, -1))

tensor([[[ 1.3420, -0.6658, -0.2153,  0.3321],
         [-0.9340,  1.6575,  1.0078,  1.3923],
         [ 1.2803,  0.8974,  0.0036, -0.2383]],

        [[-1.8412, -0.6197, -1.1801, -0.8510],
         [-0.8261, -0.6701,  0.9620, -1.6931],
         [-0.7102,  2.8641,  1.0953,  0.7799]]])
tensor([[ 1.3420, -0.6658, -0.2153,  0.3321, -0.9340,  1.6575,  1.0078,  1.3923,
          1.2803,  0.8974,  0.0036, -0.2383],
        [-1.8412, -0.6197, -1.1801, -0.8510, -0.8261, -0.6701,  0.9620, -1.6931,
         -0.7102,  2.8641,  1.0953,  0.7799]])
tensor([[ 1.3420, -0.6658, -0.2153,  0.3321, -0.9340,  1.6575,  1.0078,  1.3923,
          1.2803,  0.8974,  0.0036, -0.2383],
        [-1.8412, -0.6197, -1.1801, -0.8510, -0.8261, -0.6701,  0.9620, -1.6931,
         -0.7102,  2.8641,  1.0953,  0.7799]])


## 2 Computation graphs and  Automatic Differentiation
### 计算图和自动差分

The concept of a computation graph is essential to efficient deep learning programming, because it allows you to not have to write the back propagation gradients yourself. A computation graph is simply a specification of how your data is combined to give you the output. Since the graph totally specifies what parameters were involved with which operations, it contains enough information to compute derivatives. This probably sounds vague, so lets see what is going on using the fundamental class of Pytorch: autograd.Variable.

First, think from a programmers perspective. What is stored in the torch.Tensor objects we were creating above? Obviously the data and the shape, and maybe a few other things. But when we added two tensors together, we got an output tensor. All this output tensor knows is its data and shape. It has no idea that it was the sum of two other tensors (it could have been read in from a file, it could be the result of some other operation, etc.)

The Variable class keeps track of how it was created. Lets see it in action.

In [25]:
## variables wrap tensor obj
x = autograd.Variable(torch.Tensor([1,2,3]), requires_grad=True)
print(x.data)
y = autograd.Variable(torch.Tensor([4,5,6]), requires_grad=True)
print(y.data)
z = x+y
print(z.data)
print(z.grad_fn)

tensor([1., 2., 3.])
tensor([4., 5., 6.])
tensor([5., 7., 9.])


所以当创建z时，z知道自己不是来自文件，也不是乘积的结果或者指数结果。 如果你following z.grad_fn 就会找到x,y

In [26]:
## sum all the entries in z
s = z.sum()
print(s)
print(s.grad_fn)

tensor(21., grad_fn=<SumBackward0>)


So now, what is the derivative of this sum with respect to the first component of x?  In math, we want
$$ \frac{\partial s}{\partial x_0} $$
Well, s knows that it was created as a sum of the tensor z.  z knows that it was the sum x + y.
So 
$$ s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$} $$
And so s contains enough information to determine that the derivative we want is 1!

那么，s关于x的导数是什么呢？   
s知道 它是由张量z 的和得来的，z是由 x+y 得到的，所 s = x0+y0 + x1 +y1 + x2+y2
所以 s包含了足够的信息去求得关于x的导数 1.

Of course this glosses over the challenge of how to actually compute that derivative.  The point here is that s is carrying along enough information that it is possible to compute it.  In reality, the developers of Pytorch program the sum() and + operations to know how to compute their gradients, and run the back propagation algorithm.  An in-depth discussion of that algorithm is beyond the scope of this tutorial.

Lets have Pytorch compute the gradient, and see that we were right: (note if you run this block multiple times, the gradient will increment. That is because Pytorch accumulates the gradient into the .grad property, since for many models this is very convenient.)

我们来使用pytorch 来计算梯度。 

In [27]:
s.backward() ## calling backward() on any variable will run backprop starting fron it

In [30]:
print(x.grad)
print(y.grad)

tensor([1., 1., 1.])
tensor([1., 1., 1.])


In [43]:
x = torch.randn((2,2))
y = torch.randn((2,2))
z = x+y  ## these are tensor types, and backprop would not be possible

var1_x = autograd.Variable(x)  ## 
var1_y = autograd.Variable(y)
var1_z = var1_x + var1_y
print(var1_z.grad_fn) ## has no information to backprop since var1_x and var1_y didn't enable the parameter  'requires_grad=True' 

var_x = autograd.Variable(x, requires_grad=True)
var_y = autograd.Variable(y, requires_grad=True)
var_z = var_x + var_y ## var_z contains enough information to compute gradients
print(var_z.grad_fn)

var_z_data = var_z.data ## get the wrapped tensor object out of var z
new_var_z = autograd.Variable(var_z_data) ## re-warp the tensor in a new variable

## does new_var_z have information to backprop to x and y?
## No！
print(new_var_z.grad_fn)

## and how could it? we ranke the tensor out of var_z(that is waht var_z.data is). 
## this tensor doesn't know anything about how it was computed. we pass it into new_var_z
## and this is all information new_var_z gets.  if var_z_data didn't know how it was computed,
## there is no way new_var_z will. In essence, we have broken the variable away from its past history.

new_var_z1 = autograd.Variable(var_z_data,  requires_grad=True) 
print(new_var_z1.grad_fn)

None
None
None


Here is the basic, extremely important rule for computing with autograd.Variables (note this is more general than Pytorch. There is an equivalent object in every major deep learning toolkit):

If you want the error from your loss function to backpropogate to a component of your network, you MUST NOT break the Variable chain from that component to your loss Variable. If you do, the loss will have no idea your component exists, and its parameters can't be updated.

I say this in bold, because this error can creep up on you in very subtle ways (I will show some such ways below), and it will not cause your code to crash or complain, so you must be careful.

## 3 Deep learning building blocks: affine maps, non-linearities and objectives

Deep learning consists of composing linearities with non-linearities in clever ways. The introduction of non-linearities allows for powerful models. In this section, we will play with these core components, make up an objective function, and see how the model is trained.

### Affine Maps  映射

One of the core workhorses of deep learning is the affine map, which is a function $f(x)$ where
$$ f(x) = Ax + b $$ for a matrix $A$ and vectors $x, b$.  The parameters to be learned here are $A$ and $b$.  Often, $b$ is refered to as the *bias* term.

Pytorch and most other deep learning frameworks do things a little differently than traditional linear algebra. It maps the rows of the input instead of the columns. That is, the  ii 'th row of the output below is the mapping of the  ii 'th row of the input under  AA , plus the bias term. Look at the example below.

In [47]:
lin = nn.Linear(5,3) ## maps from R^5 to R^3， parameters A,b
data = autograd.Variable(torch.randn(2,5)) ## data is 2*5, A maps from 5 to 3
print(data)
print(lin(data))

tensor([[-1.0874, -2.1025,  2.7931,  0.3121, -0.0492],
        [-0.0570, -0.0098,  0.2955, -0.2459,  2.2788]])
tensor([[ 0.1406,  1.9016,  2.0961],
        [-0.7978,  0.2609, -0.8661]], grad_fn=<AddmmBackward>)


### Non-linearities
非线性
First, note the following fact, which will explain why we need non-linearities in the first place.
Suppose we have two affine maps $f(x) = Ax + b$ and $g(x) = Cx + d$.  What is $f(g(x))$?
$$ f(g(x)) = A(Cx + d) + b = ACx + (Ad + b) $$
$AC$ is a matrix and $Ad + b$ is a vector, so we see that composing affine maps gives you an affine map.

From this, you can see that if you wanted your neural network to be long chains of affine compositions, that this adds no new power to your model than just doing a single affine map.

If we introduce non-linearities in between the affine layers, this is no longer the case, and we can build much more powerful models.

There are a few core non-linearities.  $\tanh(x), \sigma(x), \text{ReLU}(x)$ are the most common.
You are probably wondering: "why these functions?  I can think of plenty of other non-linearities."
The reason for this is that they have gradients that are easy to compute, and computing gradients is essential for learning.  For example
$$ \frac{d\sigma}{dx} = \sigma(x)(1 - \sigma(x)) $$

A quick note: although you may have learned some neural networks in your intro to AI class where $\sigma(x)$ was the default non-linearity, typically people shy away from it in practice.  This is because the gradient *vanishes* very quickly as the absolute value of the argument grows.  Small gradients means it is hard to learn.  Most people default to tanh or ReLU.

In [4]:
## in pytorch most non-linearities are in torch.funcitonal (we have it imported as F)
## noted that non-linearites typically don't have parameters like affine maps do.
## that is they don't have weights that are updated during trainning.

data = autograd.Variable(torch.randn(2,2))
print(data)
print(F.relu(data))

tensor([[-0.4519, -0.1661],
        [-1.5228,  0.3817]])
tensor([[0.0000, 0.0000],
        [0.0000, 0.3817]])


## Softmax and probabilities

The function $\text{Softmax}(x)$ is also just a non-linearity, but it is special in that it usually is the last operation done in a network.  This is because it takes in a vector of real numbers and returns a probability distribution.  Its definition is as follows.  Let $x$ be a vector of real numbers (positive, negative, whatever, there are no constraints).  Then the i'th component of $\text{Softmax}(x)$ is
$$ \frac{\exp(x_i)}{\sum_j \exp(x_j)} $$
It should be clear that the output is a probability distribution: each element is non-negative and the sum over all components is 1.

You could also think of it as just applying an element-wise exponentiation operator to the input to make everything non-negative and then dividing by the normalization constant.

In [19]:
## softmax is also in torch.functional
data = autograd.Variable(torch.randn(5))
print(data)
print(F.softmax(data, dim=0))  # dim 沿着哪个维度做softmax
print(F.softmax(data, dim=0).sum())
print(F.log_softmax(data, dim=0))

tensor([ 1.2381, -1.3459,  0.5119, -0.6933, -0.1668])
tensor([0.5129, 0.0387, 0.2481, 0.0744, 0.1259])
tensor(1.0000)
tensor([-0.6676, -3.2516, -1.3938, -2.5990, -2.0724])


## Objective Functions
目标函数是我们神经网络训练 最小化的目标。（通常称作损失函数或者cost function）
选择训练实例，在神经网络中运行，然后计算输出损失。模型的参数通过损失函数的导数来更新。
The objective function is the function that your network is being trained to minimize (in which case it is often called a loss function or cost function). This proceeds by first choosing a training instance, running it through your neural network, and then computing the loss of the output. The parameters of the model are then updated by taking the derivative of the loss function. Intuitively, if your model is completely confident in its answer, and its answer is wrong, your loss will be high. If it is very confident in its answer, and its answer is correct, the loss will be low.

损失函数最小化的思想是你的神经网络希望泛化的好（在测试样本上有较小的损失）。
The idea behind minimizing the loss function on your training examples is that your network will hopefully generalize well and have small loss on unseen examples in your dev set, test set, or in production. An example loss function is the negative log likelihood loss, which is a very common objective for multi-class classification. For supervised multi-class classification, this means training the network to minimize the negative log probability of the correct output (or equivalently, maximize the log probability of the correct output).

## 4 Optimization and Training
训练和优化

So what we can compute a loss function for an instance?  What do we do with that?
We saw earlier that autograd.Variable's know how to compute gradients with respect to the things that were used to compute it.  Well, since our loss is an autograd.Variable, we can compute gradients with respect to all of the parameters used to compute it!  Then we can perform standard gradient updates.  Let $\theta$ be our parameters, $L(\theta)$ the loss function, and $\eta$ a positive learning rate.  Then:

$$ \theta^{(t+1)} = \theta^{(t)} - \eta \nabla_\theta L(\theta) $$

There are a huge collection of algorithms and active research in attempting to do something more than just this vanilla gradient update.  Many attempt to vary the learning rate based on what is happening at train time.  You don't need to worry about what specifically these algorithms are doing unless you are really interested.  Torch provies many in the torch.optim package, and they are all completely transparent.  Using the simplest gradient update is the same as the more complicated algorithms.  Trying different update algorithms and different parameters for the update algorithms (like different initial learning rates) is important in optimizing your network's performance.  Often, just replacing vanilla SGD with an optimizer like Adam or RMSProp will boost performance noticably.

### Example: Logistic Regression Bag-of-Words classifier
Our model will map a sparse BOW representation to log probabilities over labels.  We assign each word in the vocab an index.  For example, say our entire vocab is two words "hello" and "world", with indices 0 and 1 respectively.
The BoW vector for the sentence "hello hello hello hello" is
$$ \left[ 4, 0 \right] $$
For "hello world world hello", it is 
$$ \left[ 2, 2 \right] $$
etc.
In general, it is
$$ \left[ \text{Count}(\text{hello}), \text{Count}(\text{world}) \right] $$

Denote this BOW vector as $x$.
The output of our network is:
$$ \log \text{Softmax}(Ax + b) $$
That is, we pass the input through an affine map and then do log softmax.

In [4]:
data = [('me gusta comer en la cafeteria'.split(), "SPANISH"),
        ('Given it to me'.split(), 'ENGLISH'),
        ('No creo que sea una buena idea'.split(), 'SPANISH'),
        ('No it is not a good idea to get lost at sea'.split(), 'ENGLISH')]

test_data = [("Yo creo que si".split(), 'SPANISH'),
             ('it is lost on me'.split(), 'ENGLISH')]

## word_to_ix maps each word in the vocab to a unique integer, which will be
## its index into the bag of words vector
word_to_ix = {}
for sent, _ in data+test_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS=2

{'me': 0, 'gusta': 1, 'comer': 2, 'en': 3, 'la': 4, 'cafeteria': 5, 'Given': 6, 'it': 7, 'to': 8, 'No': 9, 'creo': 10, 'que': 11, 'sea': 12, 'una': 13, 'buena': 14, 'idea': 15, 'is': 16, 'not': 17, 'a': 18, 'good': 19, 'get': 20, 'lost': 21, 'at': 22, 'Yo': 23, 'si': 24, 'on': 25}


In [5]:
class BoWClassifier(nn.Module):##继承 nn.Module
    
    def __init__(self, num_labels, vocab_size):
        ## calls the init function of nn.Module, Dont get confused by syntax
        ## just always do it in an nn.Module
        super(BoWClassifier, self).__init__()
        ## Define the parameter that you will need,
        ## In this case, we need A and b, the parameters of the affine mapping.
        ## Torch defines nn.Linear() which provides that affine map
        ## make sure you understand why the input dimesion is vocab_size
        ## and the output is num_labels.
        self.linear = nn.Linear(vocab_size, num_labels) ## from R^vocab_size to R^num_labels
    
    def forward(self, bow_vec):
        ## pass the input through the linear layer,
        ## then pass that through log_softmax
        ## many non-linearities and other functions are in torch.nn.functional
        return F.log_softmax(self.linear(bow_vec)) 
        

In [6]:
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1,-1)

def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

In [7]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

## the model knows its parameters. The first output below is A, the second is b.
## whenever you assign a component to a class variable in the __init__ function of a module
## which was done with the line self.linear = nn.linear()
## then through some python magic from pytorch devs, your model will store knowledge of the nn.Linear's parameters.

for param in model.parameters():
    print(param)

Parameter containing:
tensor([[ 0.1011, -0.0866, -0.0380,  0.0921, -0.1846,  0.1176, -0.0403,  0.0998,
          0.0273, -0.0240,  0.0544,  0.0097,  0.0716, -0.0764, -0.0143, -0.0177,
          0.0284, -0.0008,  0.1714,  0.0610, -0.0730, -0.1184, -0.0329, -0.0846,
         -0.0628,  0.0094],
        [ 0.1169,  0.1066, -0.1917,  0.1216,  0.0548,  0.1860,  0.1294, -0.1787,
         -0.1865, -0.0946,  0.1722, -0.0327,  0.0839, -0.0911,  0.1924, -0.0830,
          0.1471,  0.0023, -0.1033,  0.1008, -0.1041,  0.0577, -0.0566, -0.0215,
         -0.1885, -0.0935]], requires_grad=True)
Parameter containing:
tensor([ 0.1064, -0.0477], requires_grad=True)


In [8]:
## to run the model, pass in a BOW vector.but wrapped in a autograd.Variable
sample = data[0]
bow_vector = make_bow_vector(sample[0], word_to_ix)

log_probs = model(autograd.Variable(bow_vector))
print(log_probs)

tensor([[-0.8195, -0.5810]], grad_fn=<LogSoftmaxBackward>)


C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Which of the above values corresponds to the log probability of ENGLISH, and which to SPANISH? We never defined it, but we need to if we want to train the thing.

In [9]:
label_to_ix = {'SPANISH':0,'ENGLISH':1}

So lets train! To do this, we pass instances through to get log probabilities, compute a loss function, compute the gradient of the loss function, and then update the parameters with a gradient step. Loss functions are provided by Torch in the nn package. nn.NLLLoss() is the negative log likelihood loss we want. It also defines optimization functions in torch.optim. Here, we will just use SGD.

Note that the input to NLLLoss is a vector of log probabilities, and a target label. It doesn't compute the log probabilities for us. This is why the last layer of our network is log softmax. The loss function nn.CrossEntropyLoss() is the same as NLLLoss(), except it does the log softmax for you.

In [10]:
# To run on test data before we train, just to see a before-and-after
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    #print(bow_vec)
    log_probs = model(bow_vec)
    print(log_probs)

print(next(model.parameters())[:, word_to_ix['creo']])


tensor([[-0.6250, -0.7662]], grad_fn=<LogSoftmaxBackward>)
tensor([[-0.5870, -0.8119]], grad_fn=<LogSoftmaxBackward>)
tensor([0.0544, 0.1722], grad_fn=<SelectBackward>)


C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [11]:
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## usually you want to pass over the trainning data several times.
## 100 is much bigger than on a real data set, but real datasets have 
## more than two instances.Usually somewhere between 5 and 30 ephochs is reasonable.
for epoch in range(100):
    for instance, label in data:
        ## step1: remember that pytorch accumulates gradients. 
        ## we need to clear them out before each instance
        model.zero_grad()
        
        ## step2: Make our BOW vector and also we must wrap the target in a 
        ## Variable as an integer. For example, if the target is  SPANISH,
        ## then we wrap the integer 0. The loss function then knows that the 0th 
        ## element of the log probability is the log probability corresponding to SPANISH.
        bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
        target = autograd.Variable(make_target(label, label_to_ix))
        
        ## step3: Run our forward pass
        log_probs = model(bow_vec)
        
        ## step4: compute the loss, gradients, and update the parameters by calling
        ## optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:
for instance, label in test_data:
    bow_vec = autograd.Variable(make_bow_vector(instance, word_to_ix))
    log_probs = model(bow_vec)
    print(log_probs)
print(next(model.parameters())[:, word_to_ix['creo']])

tensor([[-0.1210, -2.1721]], grad_fn=<LogSoftmaxBackward>)
tensor([[-2.7767, -0.0643]], grad_fn=<LogSoftmaxBackward>)
tensor([ 0.5004, -0.2738], grad_fn=<SelectBackward>)


C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


 we got the right answer! You  can see that the log probability for Spanish is much higher in the first example, and the log probablity for English is much higher in the second for the test data as it shold be.
 
 Now you see how to make a pytorch component, pass some data through it an d do gradient updates, we are ready to dig deeper into what deep NLP has to offer.

## 6. Word Embeddings: Encoding Lexical Semantics

词嵌入是稠密向量。
在NLP中，特征就是单词，但是在计算机上如何表达一个单词呢？
我们可以以ASCII码来代表，但是这个仅仅告诉你这个单词是什么，并不能代表更多的意义。

Word embeddings are dense vectors of real numbers, one per word in your vocabulary.
In NLP, it is almost always the case that your features are words!  But how should you represent a word in a computer?
You could store its ascii character representation, but that only tells you what the word *is*, it doesn't say much about what it *means* (you might be able to derive its part of speech from its affixes, or properties from its capitalization, but not much).  Even more, in what sense could you combine these representations?
We often want dense outputs from our neural networks, where the inputs are $|V|$ dimensional, where $V$ is our vocabulary, but often the outputs are only a few dimensional (if we are only predicting a handful of labels, for instance).  How do we get from a massive dimensional space to a smaller dimensional space?

How about instead of ascii representations, we use a one-hot encoding?  That is, we represent the word $w$ by
$$ \overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements} $$
where the 1 is in a location unique to $w$.  Any other word will have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how huge it is.  It basically treats all words as independent entities with no relation to each other.  What we really want is some notion of *similarity* between words.  Why?  Let's see an example.


Suppose we are building a language model.  Suppose we have seen the sentences
* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data.
Now suppose we get a new sentence never before seen in our training data:
* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much better if we could use the following two facts:
* We have seen mathematician and physicist in the same role in a sentence.  Somehow they have a semantic relation.
* We have seen mathematician in the same role in this new unseen sentence as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen sentence?  This is what we mean by a notion of similarity: we mean *semantic similarity*, not simply having similar orthographic representations.  It is a technique to combat the sparsity of linguistic data, by connecting the dots between what we have seen and what we haven't.  This example of course relies on a fundamental linguistic assumption: that words appearing in similar contexts are related to each other semantically.  This is called the [distributional hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics).

### Geting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode semantic similarity in words? Maybe we think up some semantic attributes. For example, we see that both mathematicians ans physicists can run, so maybe we give these words a high score for the 'is able to run' semantic attribute. Think of some other attributes and imagine what you might score some common words on those attributes.

if each attribute is a dimension, then we might give each word a vector, like this:
$$ q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
\overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right] $$
$$ q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
\overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right] $$

Then we can get a measure of similarity between these words by doing:
$$ \text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician} $$

Although it is more common to normalize by the lengths:
$$ \text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
{\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi) $$
Where $\phi$ is the angle between the two vectors.  That way, extremely similar words (words whose embeddings point in the same direction) will have similarity 1.  Extremely dissimilar words should have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this section as a special case of these new vectors we have defined, where each word basically has similarity 0, and we gave each word some unique semantic attribute. These new vectors are dense, which is to say their entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of different semantic attributes that might be relevant to determining similarity, and how on earth would you set the values of the different attributes? Central to the idea of deep learning is that the neural network learns representations of the features, rather than requiring the programmer to design them herself. So why not just let the word embeddings be parameters in our model, and then be updated during training? This is exactly what we will do. We will have some latent semantic attributes that the network can, in principle, learn. Note that the word embeddings will probably not be interpretable. That is, although with our hand-crafted vectors above we can see that mathematicians and physicists are similar in that they both like coffee, if we allow a neural network to learn the embeddings and see that both mathematicians and physicisits have a large value in the second dimension, it is not clear what that means. They are similar in some latent semantic dimension, but this probably has no interpretation to us.

In summary, **word embeddings are a representation of the *semantics* of a word, efficiently encoding semantic information that might be relevant to the task at hand**.  You can embed other things too: part of speech tags, parse trees, anything!  The idea of feature embeddings is central to the field.

### Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes about how to use embeddings in pytorch and in deep learining programming in general. Similar to how we defined a unique index for each word when making one-hot vectors, we also need to define an index fo reach word when using embeddings.  These will be keys into a lookup table.  That is, embeddings are stored as a $|V| \times D$ matrix, where $D$ is the dimensionality of the embeddings, such that the word assigned index $i$ has its embedding stored in the $i$'th row of the matrix.  In all of my code, the mapping from words to indices is a dictionary named word_to_ix.

The module that allows you to use embeddings is **torch.nn.Embedding**, which takes two arguments: the vocabulary size, and the dimensionality of the embeddings.

To index into this table, you must use torch.LongTensor (since the indices are integers, not floats).

In [13]:
word_to_ix = {'hello':0, "word":1}
embeds = nn.Embedding(2,5) ## 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

tensor([[ 1.8213, -0.1814, -0.9515,  0.4057, -1.5164]],
       grad_fn=<EmbeddingBackward>)


### An Example: N-Gram Language Modeling
Recall that in an n-gram language model, given a sequence of words $w$, we want to compute
$$ P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} ) $$
Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training examples and update the parameters with backpropagation.

In [62]:
CONTEXT_SIZE=2
EMBEDDING_DIM=10
# we will use shakespeare sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
## we should tokenize the input, but we will ignore that for now build a list of 
## tuples. Each tuple is ([word_i-2, word_i-1], target_word)
trigrams = [([test_sentence[i], test_sentence[i+1]], test_sentence[i+2]) for i in range(len(test_sentence) -2)]
print(trigrams)

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege'), (['shall', 'besiege'], 'thy'), (['besiege', 'thy'], 'brow,'), (['thy', 'brow,'], 'And'), (['brow,', 'And'], 'dig'), (['And', 'dig'], 'deep'), (['dig', 'deep'], 'trenches'), (['deep', 'trenches'], 'in'), (['trenches', 'in'], 'thy'), (['in', 'thy'], "beauty's"), (['thy', "beauty's"], 'field,'), (["beauty's", 'field,'], 'Thy'), (['field,', 'Thy'], "youth's"), (['Thy', "youth's"], 'proud'), (["youth's", 'proud'], 'livery'), (['proud', 'livery'], 'so'), (['livery', 'so'], 'gazed'), (['so', 'gazed'], 'on'), (['gazed', 'on'], 'now,'), (['on', 'now,'], 'Will'), (['now,', 'Will'], 'be'), (['Will', 'be'], 'a'), (['be', 'a'], "totter'd"), (['a', "totter'd"], 'weed'), (["totter'd", 'weed'], 'of'), (['weed', 'of'], 'small'), (['of', 'small'], 'worth'), (['small', 'worth'], 'held:'), (['worth', 'held:'], 'Then'), (['held:', 'Then'], 'being'), (['Then', 'being'], 'asked,'), (['being', 'asked,'], 'wher

In [63]:
vocab = set(test_sentence)
word_to_ix =  {word: i for i, word in enumerate(vocab)}

In [64]:
word_to_ix

{"'This": 79,
 'And': 59,
 'How': 63,
 'If': 91,
 'Proving': 1,
 'Shall': 95,
 'Then': 65,
 'This': 4,
 'Thy': 89,
 'To': 56,
 'Were': 14,
 'When': 57,
 'Where': 85,
 'Will': 60,
 'a': 87,
 'all': 26,
 'all-eating': 9,
 'an': 51,
 'and': 77,
 'answer': 88,
 'art': 22,
 'asked,': 36,
 'be': 40,
 'beauty': 49,
 "beauty's": 19,
 'being': 18,
 'besiege': 33,
 'blood': 17,
 'brow,': 78,
 'by': 70,
 'child': 71,
 'cold.': 8,
 'couldst': 0,
 'count,': 45,
 'days;': 28,
 'deep': 5,
 "deserv'd": 34,
 'dig': 42,
 "excuse,'": 54,
 'eyes,': 64,
 'fair': 15,
 "feel'st": 23,
 'field,': 82,
 'forty': 6,
 'gazed': 11,
 'held:': 46,
 'his': 93,
 'in': 58,
 'it': 73,
 'lies,': 66,
 'livery': 25,
 'lusty': 43,
 'made': 76,
 'make': 94,
 'mine': 62,
 'more': 55,
 'much': 61,
 'my': 69,
 'new': 81,
 'now,': 68,
 'of': 13,
 'old': 37,
 'old,': 12,
 'on': 20,
 'own': 41,
 'praise': 32,
 'praise.': 50,
 'proud': 38,
 'say,': 7,
 'see': 39,
 'shall': 86,
 'shame,': 90,
 'small': 24,
 'so': 72,
 'succession': 6

In [65]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size*embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [67]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for each in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:
        ## step1. prepare the inputs to be passed to the model(i.e, turn the
        ## word into integer indices and wrap them in variables)
        context_idxs = list(map(lambda w: word_to_ix[w], context))
        context_var = autograd.Variable(torch.LongTensor(context_idxs))
        
        ##step2. Recall that torch *accumulates* gradients, Before passing 
        ## in a new instance, you need to zero out the gradients from the old instance
        model.zero_grad()
        
        ## step3. Run the forward pass, getting log probabilities over next words
        log_probs = model(context_var)
        
        ## step4. Compute your loss function.(Again, torch wants the target
        ## word wrapped in a varaible)
        print(log_probs)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        
        ## step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


tensor([[-4.1535, -4.7271, -4.5265, -4.8250, -5.2913, -4.8001, -4.7563, -4.3589,
         -4.5569, -5.0000, -4.5919, -4.6016, -3.8724, -4.6865, -4.4513, -4.5974,
         -4.4559, -4.3683, -4.9112, -4.9796, -4.6693, -4.4323, -4.4958, -4.7482,
         -4.5464, -4.7935, -4.1770, -4.5300, -4.6097, -4.4339, -4.6342, -4.5058,
         -4.5779, -5.1861, -5.0929, -4.9019, -4.3471, -4.7208, -4.6505, -4.3828,
         -4.4953, -4.8747, -4.6526, -4.6505, -4.5770, -4.1396, -4.8143, -4.7202,
         -3.8189, -4.7477, -4.5296, -5.1305, -4.1948, -5.3009, -4.5195, -4.9329,
         -4.6298, -4.7332, -5.2154, -4.4720, -4.3263, -4.0894, -4.6735, -4.3175,
         -4.7431, -4.5124, -4.4448, -4.9108, -4.6160, -4.4150, -4.2865, -4.5649,
         -4.6016, -4.8413, -4.6733, -4.5784, -4.5009, -4.7829, -4.4192, -4.3826,
         -4.6125, -4.2066, -4.1889, -4.7110, -4.7966, -4.9974, -4.7365, -4.9263,
         -4.1269, -5.1411, -4.1205, -5.1835, -4.7849, -4.9563, -4.4913, -4.4717,
         -4.6903]], grad_fn=

tensor([[-4.5642, -4.9734, -4.6590, -4.0981, -5.0925, -4.5586, -4.8756, -4.5454,
         -4.4410, -4.9843, -4.4756, -4.4617, -4.6770, -4.6354, -4.8298, -4.5762,
         -4.4609, -4.6052, -4.6128, -4.8623, -4.9684, -4.6155, -4.6868, -4.4995,
         -4.2050, -4.7209, -4.2902, -4.5014, -4.7405, -4.4189, -4.7569, -4.5083,
         -4.4687, -4.8304, -4.5272, -4.8432, -4.0679, -4.6600, -4.4688, -4.4986,
         -4.8229, -4.5787, -4.7456, -4.8466, -4.3293, -4.4338, -4.8561, -4.8452,
         -3.8777, -4.6771, -4.4917, -5.0734, -4.1730, -5.1154, -4.1809, -4.6778,
         -4.8992, -5.2550, -4.8105, -4.2585, -4.1284, -4.3396, -4.6805, -4.1665,
         -4.5951, -4.6412, -4.7160, -4.8168, -4.6711, -4.4795, -4.1839, -4.5174,
         -4.5393, -4.8923, -4.5417, -4.3105, -4.7982, -4.8291, -4.3599, -4.6669,
         -4.5890, -4.5830, -3.9326, -4.4248, -4.7140, -5.2277, -4.8159, -4.9659,
         -4.1747, -4.7227, -4.6464, -5.1300, -5.0803, -4.4382, -4.3610, -4.6073,
         -5.1159]], grad_fn=

tensor([[-4.5826, -4.4790, -4.5652, -4.4775, -4.3935, -4.7645, -5.1243, -4.7675,
         -4.6580, -4.7877, -4.5258, -4.4814, -4.5146, -4.7702, -4.4978, -4.3120,
         -4.5079, -4.5430, -4.4290, -4.6391, -4.5608, -4.9224, -4.8299, -4.8056,
         -4.3024, -4.7414, -4.4527, -4.5504, -4.5275, -4.9597, -4.5342, -4.7845,
         -4.6569, -4.4389, -4.6353, -4.6442, -4.4582, -4.7323, -4.5511, -4.5001,
         -4.4099, -4.5704, -4.5346, -4.4617, -4.5275, -4.4655, -4.8202, -4.7588,
         -4.0814, -4.4274, -4.4696, -4.7105, -4.7277, -4.9213, -4.4182, -4.4811,
         -4.5434, -4.7489, -4.7549, -4.6503, -4.4627, -4.2332, -4.4248, -4.1861,
         -4.6952, -4.7007, -4.7746, -4.6508, -4.4427, -4.3017, -4.6757, -4.6320,
         -4.3010, -4.4125, -4.4694, -4.7264, -4.5049, -4.6691, -4.7930, -4.3099,
         -4.8215, -4.6589, -4.3502, -4.6215, -4.4163, -4.6184, -4.7660, -4.6908,
         -4.6751, -4.9301, -4.1694, -5.0691, -4.8859, -4.6938, -4.6567, -4.5600,
         -4.6743]], grad_fn=

         -4.4711]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.3024, -4.6914, -4.5309, -4.2861, -5.2280, -4.6102, -5.2940, -4.3439,
         -4.5935, -4.9991, -4.4824, -4.6780, -4.4557, -4.2483, -4.7241, -4.3772,
         -4.1357, -4.8326, -4.5646, -4.9061, -5.1130, -4.2406, -5.1403, -5.1363,
         -4.5001, -4.4165, -4.2276, -4.6811, -4.8651, -4.5385, -4.7243, -4.2191,
         -4.8084, -4.7582, -4.6094, -4.5988, -4.3131, -4.8040, -4.8151, -4.7741,
         -4.7284, -4.8813, -4.8678, -4.8199, -4.4850, -4.6942, -4.9461, -4.6934,
         -4.0260, -4.4605, -4.6155, -4.5971, -4.3392, -4.7266, -4.8217, -4.2878,
         -4.4724, -4.3804, -5.2366, -4.5428, -4.2924, -3.9919, -4.6977, -3.9898,
         -4.4534, -4.8917, -4.9755, -4.8707, -4.7223, -4.4396, -4.4928, -4.6398,
         -4.4690, -4.3019, -4.5096, -4.4701, -4.4456, -4.3691, -4.3832, -5.1406,
         -4.5197, -4.3650, -4.0233, -4.6317, -4.5511, -4.8220, -4.8095, -4.7420,
         -4.6450, -4.4879, -4.3444, -5.2139, -4.9487, -4.94

...

tensor([[-4.4266, -4.5650, -5.0318, -3.8843, -5.0946, -5.1706, -4.7312, -4.5814,
         -4.4439, -5.2463, -4.4278, -4.6157, -4.6972, -4.4640, -4.3787, -4.7372,
         -4.5532, -4.6728, -4.6662, -4.7739, -4.7881, -4.5475, -4.4952, -4.4534,
         -4.4229, -4.9797, -4.1262, -4.6724, -4.7323, -4.9619, -4.4889, -4.8080,
         -4.5135, -4.9672, -4.9170, -4.8002, -4.3280, -4.3545, -4.7753, -4.3567,
         -4.9057, -4.9884, -4.7477, -4.6753, -4.8564, -4.2604, -4.8594, -4.8301,
         -3.8003, -4.9151, -4.3326, -4.7608, -4.0820, -5.3745, -4.3962, -4.9441,
         -4.4634, -5.1083, -4.7813, -4.3460, -4.4247, -4.5169, -4.5932, -3.9784,
         -4.3564, -4.3734, -4.5831, -4.2522, -4.5137, -3.9515, -4.5656, -4.6380,
         -4.7242, -4.6500, -4.8563, -4.4857, -4.8329, -4.2620, -4.8276, -4.4014,
         -4.8381, -4.3292, -4.1278, -4.5389, -4.6672, -5.1897, -4.6528, -4.8390,
         -4.4017, -5.2977, -4.3859, -5.1011, -5.3500, -4.2617, -4.4070, -4.4129,
         -4.9637]], grad_fn=

tensor([[-4.7985, -4.4727, -4.9153, -4.1165, -4.6512, -5.1438, -4.7983, -4.8545,
         -4.7388, -4.6965, -4.6356, -4.5054, -4.3615, -4.2701, -4.6376, -4.5007,
         -4.4695, -4.2940, -4.4285, -4.6816, -4.9323, -4.5439, -4.8244, -4.7852,
         -4.0564, -4.8918, -4.4196, -4.6039, -4.6830, -4.3317, -4.6244, -4.4238,
         -4.9851, -4.9940, -4.6720, -4.4004, -4.9114, -4.3574, -4.6303, -4.5774,
         -4.7643, -4.8612, -4.8511, -4.7753, -4.5302, -4.6903, -4.8244, -4.9426,
         -4.0436, -4.6302, -4.3083, -5.1701, -4.3590, -5.0872, -4.4283, -4.4293,
         -5.1408, -4.9345, -4.6574, -4.4020, -4.3897, -4.3895, -4.3642, -4.3487,
         -4.4575, -4.1369, -4.5721, -4.7638, -4.1849, -4.6869, -4.5073, -4.5400,
         -4.2163, -4.4457, -4.3952, -4.4767, -4.5464, -4.5746, -4.3528, -4.7840,
         -4.7285, -4.1820, -4.2413, -4.2782, -4.5828, -5.0350, -4.5897, -4.7176,
         -4.8664, -4.5943, -4.4775, -5.0699, -4.7857, -4.9893, -4.8831, -4.9931,
         -4.4346]], grad_fn=

         -4.6994]], grad_fn=<LogSoftmaxBackward>)
tensor([[-4.4689, -4.5800, -4.5502, -3.8653, -4.7727, -4.9067, -4.4671, -4.8034,
         -4.5989, -5.3472, -4.9663, -4.3787, -4.8709, -4.6724, -5.0083, -4.4691,
         -4.6091, -4.7717, -4.8602, -4.6266, -4.9648, -4.7911, -4.5669, -4.9244,
         -4.3820, -4.6262, -4.0284, -4.6120, -4.9220, -4.7038, -4.3816, -4.7231,
         -4.6346, -5.0976, -5.1005, -4.7546, -4.5589, -4.2599, -4.4201, -4.6136,
         -4.8612, -4.9399, -4.3277, -4.2409, -4.6212, -4.2671, -4.7707, -4.5049,
         -3.8464, -4.5518, -4.2704, -5.1242, -4.6005, -4.9336, -4.2843, -4.9224,
         -4.7558, -4.7667, -4.7091, -4.5274, -4.2066, -4.7435, -4.1777, -4.4683,
         -4.7233, -4.6065, -4.8703, -4.5303, -4.1775, -4.7781, -4.6413, -4.5293,
         -4.4445, -4.7410, -4.1397, -4.7723, -4.2874, -4.8412, -4.4512, -4.3935,
         -4.5132, -4.4965, -4.4835, -4.4582, -4.8572, -4.5349, -4.9378, -4.5085,
         -4.5977, -5.1196, -4.2115, -4.5457, -4.2860, -4.67

tensor([[-4.6896, -4.5164, -4.5994, -4.1144, -4.6799, -5.0974, -5.3335, -4.7370,
         -4.3933, -4.7649, -4.8858, -4.4045, -4.5109, -4.5707, -4.7084, -4.3842,
         -4.5120, -4.6537, -4.3693, -4.4085, -4.6028, -4.4457, -4.6073, -4.8915,
         -4.2559, -4.6346, -4.8018, -4.7143, -4.6608, -4.6058, -4.6392, -4.8105,
         -4.7010, -4.3457, -4.2574, -4.5532, -5.0375, -4.4098, -4.4088, -4.4645,
         -4.4940, -4.7718, -4.7200, -4.3205, -4.6105, -5.2327, -4.5786, -4.6536,
         -4.0021, -4.1158, -4.2235, -4.7350, -4.7139, -4.4277, -4.4607, -4.1502,
         -4.6033, -5.0292, -4.7287, -4.2468, -4.8062, -4.2930, -4.3411, -4.6525,
         -4.6695, -5.0641, -4.7579, -4.5908, -4.7538, -4.6614, -4.8869, -4.7896,
         -3.8663, -4.3057, -4.6874, -4.7964, -4.6027, -4.7225, -4.3263, -4.5434,
         -4.4454, -4.7315, -4.6825, -4.4549, -4.6741, -4.6681, -4.4830, -4.6715,
         -4.9656, -4.5454, -4.1828, -5.0442, -4.7112, -4.9690, -4.7179, -5.1119,
         -4.8647]], grad_fn=

tensor([[-4.4743, -4.7102, -4.6435, -4.1432, -5.1695, -4.6886, -5.2475, -4.8176,
         -4.4536, -4.8105, -4.7537, -4.8527, -4.5468, -4.7378, -4.3071, -4.6026,
         -4.1765, -4.6156, -5.1153, -4.5719, -4.6877, -4.6368, -4.9643, -4.9568,
         -4.8403, -4.6240, -3.9362, -4.0259, -4.8842, -4.8501, -4.4585, -5.1157,
         -5.0090, -4.6262, -4.7079, -4.7281, -4.3976, -4.8191, -4.4877, -4.2693,
         -4.5030, -4.8598, -4.6264, -4.4581, -4.5926, -4.7315, -4.7492, -4.6505,
         -3.5990, -4.2371, -4.5394, -4.7759, -4.8237, -5.3237, -4.0485, -4.8953,
         -4.3453, -5.1988, -5.1676, -4.3334, -4.4390, -4.5312, -5.0476, -4.1849,
         -4.7644, -4.7354, -4.2479, -4.8059, -4.6278, -3.9160, -4.7807, -4.5302,
         -4.2690, -4.4587, -4.6804, -4.7670, -5.0091, -4.6322, -4.5959, -4.1522,
         -4.1192, -4.6863, -3.6568, -4.6795, -4.6157, -4.9327, -4.5245, -4.9280,
         -5.1472, -5.2184, -4.1593, -5.2434, -4.8671, -4.6587, -4.9368, -4.5138,
         -4.6886]], grad_fn=

### Exercise: computing word Embeddings: continuous bag-og-words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep learning.  It is a model that tries to predict words given the context of a few words before and a few words after the target word.  This is distinct from language modeling, since CBOW is not sequential and does not have to be probabilistic.  Typcially, CBOW is used to quickly train word embeddings, and these embeddings are used to initialize the embeddings of some more complicated model.  Usually, this is referred to as *pretraining embeddings*.  It almost always helps performance a couple of percent.

The CBOW model is as follows.  Given a target word $w_i$ and an $N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$ and $w_{i+1}, \dots, w_{i+N}$, referring to all context words collectively as $C$, CBOW tries to minimize
$$ -\log p(w_i | C) = \log \text{Softmax}(A(\sum_{w \in C} q_w) + b) $$
where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some tips:
* Think about which parameters you need to define.
* Make sure you know what shape each operation expects.  Use .view() if you need to reshape.

In [73]:
CONTEXT_SIZE = 2 # 2 words to the left, 2 to the right 上下文  窗口大小
raw_text =  """We are about to study the idea of a computational process. Computational processes are abstract
beings that inhabit computers. As they evolve, processes manipulate other abstract
things called data. The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()
word_to_ix = {word: i for i, word in enumerate(set(raw_text))}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i-2], raw_text[i-1], raw_text[i+1], raw_text[i+2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


In [74]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_size, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear1 = nn.Linear(context_size*2*embedding_size, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs

In [75]:
def make_context_vector(context, word_to_ix):
    idxs = list(map(lambda x: word_to_ix[x], context))
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)
print(make_context_vector(data[0][0], word_to_ix))

def make_target_vector(target, word_to_ix):
    tensor = torch.LongTensor([word_to_ix[target]])
    return autograd.Variable(tensor)

tensor([39, 22, 14, 15])


In [76]:
losses = []
loss_function = nn.NLLLoss()
vocab = set(raw_text)
EMBEDDING_DIM = 5
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr =0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in data:
        context_var = autograd.Variable(make_context_vector(context, word_to_ix))
        model.zero_grad()
        log_probs = model(context_var)
        loss = loss_function(log_probs, autograd.Variable(torch.LongTensor([word_to_ix[target]])))
        loss.backward()
        optimizer.step()
        total_loss += loss.data
    losses.append(total_loss)
print(losses)

C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':


[tensor([226.3654]), tensor([224.8524]), tensor([223.3525]), tensor([221.8654]), tensor([220.3905]), tensor([218.9271]), tensor([217.4745]), tensor([216.0321]), tensor([214.6006]), tensor([213.1796])]


## 7 Sequence Models and Long-Short Term Memory Networks

At this point, we have seen various feed-forward networks. That is, there is no state manintained by the network at all.
This might not be the behavior we want.

Sequence model are central to NLP: they are models where there is some sort of dependence through time between your inputs.
The classical example of a sequence model is the Hidden Markov Model for part of speech tagging. Another example is the conditional random field.

HMM and CRF.

A recurrent neural network is a network that maintains some kind of state. For example, its output could be used as part of the next input, so that information can propogate along as the network passes over the sequence. 
In the case of an LSTM, for each element in the sequence, there is a corresponding **hidden state** $h_t$, which in principle can contain information from arbitrary points earlier in the sequence.
We can use the hidden state to predict words in a language model, part of speech tags, and a myriad of other things.

### LSTM's in Pytorch

Pytorch LSTM 期望所有的输入都是3维张量。每一个维度的语义都是非常重要的。
第一维是 序列自身，
第二维是最小batch（分批处理的最小批次），
第三维是输入的元素。

我们还没有讨论过 mini-batch, 目前我们先默认第二维是1.


Before getting to the example, note a few things.
Pytorch's LSTM expects all of its inputs to be 3D tensors.
The semantics of the axes of these tensors is important.
The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input.
We haven't discussed mini-batching, so lets just ignore that and assume we will always have just 1 dimension on the second axis.
If we want to run the sequence model over the sentence "The cow jumped", our input should look like
$$ 
\begin{bmatrix}
\overbrace{q_\text{The}}^\text{row vector} \\
q_\text{cow} \\
q_\text{jumped}
\end{bmatrix}
$$
Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which case the 1st axis will have size 1 also.

Let's see a quick example.

nn.LSTM?
Docstring:     
Applies a multi-layer long short-term memory (LSTM) RNN to an input
sequence.


For each element in the input sequence, each layer computes the following
function:

.. math::
    \begin{array}{ll} \\
        i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{(t-1)} + b_{hi}) \\
        f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{(t-1)} + b_{hf}) \\
        g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{(t-1)} + b_{hg}) \\
        o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{(t-1)} + b_{ho}) \\
        c_t = f_t * c_{(t-1)} + i_t * g_t \\
        h_t = o_t * \tanh(c_t) \\
    \end{array}


where :
math:$h_t$ is the hidden state at time 't', 
math:$c_t$ is the cell state at time 't', 
math:$x_t$ is the input at time `t`, 
math:$h_{(t-1)}$ is the hidden state of the layer at time `t-1` or the initial hidden state at time `0`, 
math:`i_t`,math:`f_t`, math:`g_t`,math:`o_t` are the input, forget, cell, and output gates,respectively.
math:$\sigma$ is the sigmoid function, 
math:$*$ is the Hadamard product.

In a multilayer LSTM, the input :math:`x^{(l)}_t` of the :math:`l` -th layer
(:math:`l >= 2`) is the hidden state :math:`h^{(l-1)}_t` of the previous layer multiplied by
dropout :math:`\delta^{(l-1)}_t` where each :math:`\delta^{(l-1)}_t` is a Bernoulli random
variable which is :math:`0` with probability :attr:`dropout`.

Args:
    input_size: The number of expected features in the input `x`. 输入数据的特征数量
    
    hidden_size: The number of features in the hidden state `h`.  隐含层的特征数量
    
    num_layers: Number of recurrent layers. E.g., setting ``num_layers=2`` 
        would mean stacking two LSTMs together to form a `stacked LSTM`,
        with the second LSTM taking in outputs of the first LSTM and
        computing the final results. Default: 1. LSTM的层数，默认是 1
        
    bias: If ``False``, then the layer does not use bias weights `b_ih` and `b_hh`.  
        Default: ``True``.是否使用bias
        
    batch_first: If ``True``, then the input and output tensors are provided . 
        as (batch, seq, feature). Default: ``False``.   默认数据格式（seq, batch, feature）
        
    dropout: If non-zero, introduces a `Dropout` layer on the outputs of each
        LSTM layer except the last layer, with dropout probability equal to
        :attr:`dropout`. Default: 0                       除了最后一层之外都引入dropput
        
    bidirectional: If ``True``, becomes a bidirectional LSTM. Default: ``False``  是否创建双向LSTM
    

Inputs: input, (h_0, c_0)       h_0 表示上一层的输出，c_0表示上一层的状态
    - **input** of shape `(seq_len, batch, input_size)`: tensor containing the features
      of the input sequence.
      The input can also be a packed variable length sequence.
      See :func:`torch.nn.utils.rnn.pack_padded_sequence` or
      :func:`torch.nn.utils.rnn.pack_sequence` for details.
    - **h_0** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor
      containing the initial hidden state for each element in the batch.
      If the LSTM is bidirectional, num_directions should be 2, else it should be 1.
    - **c_0** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor
      containing the initial cell state for each element in the batch.

      If `(h_0, c_0)` is not provided, both **h_0** and **c_0** default to zero.


Outputs: output, (h_n, c_n)
    - **output** of shape `(seq_len, batch, num_directions * hidden_size)`: tensor
      containing the output features `(h_t)` from the last layer of the LSTM,
      for each `t`. If a :class:`torch.nn.utils.rnn.PackedSequence` has been
      given as the input, the output will also be a packed sequence.

      For the unpacked case, the directions can be separated
      using ``output.view(seq_len, batch, num_directions, hidden_size)``,
      with forward and backward being direction `0` and `1` respectively.
      Similarly, the directions can be separated in the packed case.
    - **h_n** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor
      containing the hidden state for `t = seq_len`.

      Like *output*, the layers can be separated using
      ``h_n.view(num_layers, num_directions, batch, hidden_size)`` and similarly for *c_n*.
    - **c_n** of shape `(num_layers * num_directions, batch, hidden_size)`: tensor
      containing the cell state for `t = seq_len`.

Attributes:
    weight_ih_l[k] : the learnable input-hidden weights of the :math:`\text{k}^{th}` layer
        `(W_ii|W_if|W_ig|W_io)`, of shape `(4*hidden_size, input_size)` for `k = 0`.
        Otherwise, the shape is `(4*hidden_size, num_directions * hidden_size)`
    weight_hh_l[k] : the learnable hidden-hidden weights of the :math:`\text{k}^{th}` layer
        `(W_hi|W_hf|W_hg|W_ho)`, of shape `(4*hidden_size, hidden_size)`
    bias_ih_l[k] : the learnable input-hidden bias of the :math:`\text{k}^{th}` layer
        `(b_ii|b_if|b_ig|b_io)`, of shape `(4*hidden_size)`
    bias_hh_l[k] : the learnable hidden-hidden bias of the :math:`\text{k}^{th}` layer
        `(b_hi|b_hf|b_hg|b_ho)`, of shape `(4*hidden_size)`

.. note::
    All the weights and biases are initialized from :math:`\mathcal{U}(-\sqrt{k}, \sqrt{k})`
    where :math:`k = \frac{1}{\text{hidden\_size}}`

.. include:: cudnn_persistent_rnn.rst

Examples::

    >>> rnn = nn.LSTM(10, 20, 2)  输入数据的特征数量是10，隐藏层特征数量是20, 2层LSTM cell单元。
    >>> input = torch.randn(5, 3, 10) 输入数据是 5个（seq） 3（batch size） 10(特征数量)
    >>> h0 = torch.randn(2, 3, 20)   num_layers * num_directions, batch, hidden_size 
    >>> c0 = torch.randn(2, 3, 20)
    >>> output, (hn, cn) = rnn(input, (h0, c0))
File:           c:\users\lijie4\appdata\local\continuum\anaconda3\lib\site-packages\torch\nn\modules\rnn.py
Type:           type

In [94]:
lstm = nn.LSTM(3,3) ## Input dim is 3, output dim is 3
inputs = [autograd.Variable(torch.randn((1,3))) for _ in range(5)] ## make a sequence of lenght 5

In [95]:
## initialize the hidden state
## hidden = (h0,c0)  h0=(num_layers * num_directions, batch, hidden_size )
hidden =(autograd.Variable(torch.randn(1,1,3)), autograd.Variable(torch.randn(1,1,3)))
for i in inputs:
    ## step through the sequence one element at a time.
    ## after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1,1,-1), hidden)

## alternatively, we can do the entire sequence all at once.
## the first value returned by LSTM is all of the hidden states throughout 
## the sequence. 
## the second is just the most recent hidden state(compare the last slice of
## "out" with "hidden" as below, they are the same)
## the reason for this is that:
## "out" will give you access to all hidden states in the sequence.
## "hidden" will allow you to continue the sequence and backpropogate, by 
## passing it as an argument to the lstm at a later time.

## add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1) 
# clean out hidden state
hidden = (autograd.Variable(torch.randn(1,1,3)), autograd.Variable(torch.randn(1,1,3)))
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[ 0.4259, -0.1994,  0.1684]],

        [[ 0.1845,  0.0726,  0.1253]],

        [[-0.0046,  0.0497, -0.0056]],

        [[-0.0359,  0.2281,  0.0090]],

        [[ 0.0249,  0.0805,  0.0215]]], grad_fn=<StackBackward>)
(tensor([[[0.0249, 0.0805, 0.0215]]], grad_fn=<StackBackward>), tensor([[[0.0559, 0.1079, 0.0738]]], grad_fn=<StackBackward>))


### Example: An LSTM for part-of-speech Tagging

In this section, we will use an LSTM to get part of speech tags.
We will not use Viterbi or Forward-Backward or anything like that, but as a (challenging) exercise to the reader, think about how Viterbi could be used after you have seen what is going on.

The model is as follows: let our input sentence be $w_1, \dots, w_M$, where $w_i \in V$, our vocab.
Also, let $T$ be our tag set, and $y_i$ the tag of word $w_i$.  Denote our prediction of the tag of word $w_i$ by $\hat{y}_i$.

This is a structure prediction, model, where our output is a sequence $\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.

To do the prediction, pass an LSTM over the sentence.  Denote the hidden state at timestep $i$ as $h_i$.  Also, assign each tag a unique index (like how we had word_to_ix in the word embeddings section).
Then our prediction rule for $\hat{y}_i$ is
$$ \hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j $$
That is, take the log softmax of the affine map of the hidden state, and the predicted tag is the tag that has the maximum value in this vector.  Note this implies immediately that the dimensionality of the target space of $A$ is $|T|$.

In [96]:
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda x: to_ix[x], seq))
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)

In [97]:
training_data = [
    ("The dog ate the apple".split(), ["DET", 'NN', 'V', 'DET', 'NN']),
    ("Everybody read that book".split(), ['NN', 'V', 'DET', 'NN'])
]

word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

tag_to_ix = {'DET':0,'NN':1,"V":2}

## These will usually be more like 32 ore 64 dimensional
## we will keep them small, so we can see how the weight change as we train
EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [98]:
class LSTMagger(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        ## the LSTM takes word embeddings as input, and oupts hidden states
        ## with dimensionality hidden_dim
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        ## the linear layer that map from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        ## before we've done anything, we dont have any hidden state
        ## refer to the pytorch documentation to see exactly why they have 
        ## this dimensionality. The axes semantics are (num_layers, minibatch-size， hidden_dim)
        return (autograd.Variable(torch.zeros(1,1,self.hidden_dim)), 
                autograd.Variable(torch.zeros(1,1,self.hidden_dim)))
    
    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space)
        return tag_scores

In [99]:
model = LSTMagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [100]:
## see what the scores are before trainning
## note that element i,j of the output is the score for tag j for word i.
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
print(tag_scores)

tensor([[-1.4804, -1.3059, -0.6901],
        [-1.4338, -1.2854, -0.7235],
        [-1.4428, -1.2309, -0.7514],
        [-1.4132, -1.2215, -0.7725],
        [-1.2985, -1.1999, -0.8538]], grad_fn=<LogSoftmaxBackward>)


C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [101]:
for epoch in range(300): # normally you won't do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # step1: remember that pytorch accumulates gradients, we need to clear
        # them out before each instance
        model.zero_grad()
        # we need to clear out the hidden state of the LSTM, detaching it from
        # its history on the last instance.
        model.hidden = model.init_hidden()
        
        # step 2.Get our inputs ready for network, that is turn them into Variables
        # of word indices
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        
        # step3. Run our forward pass
        tag_scores = model(sentence_in)
        
        # step4. Compute the loss, gradients, and update the parameters by calling
        # optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [103]:
## see what the scores are after training
inputs = prepare_sequence(training_data[0][0], word_to_ix)
tag_scores = model(inputs)
## the sentence is 'the dog ate the apple'. i,j corresponds to score for tag
## j for word i. The predicted tag is the maximum scoring tag.
## here we can see the predicted sequence below is 0 1 2 0 1
## since 0 is index of the maximum value of row 1. 1 is the index of maximum
## value of row 2. etc.
## which is DET NOUN VERB DET NOUN,the correct seqence
print(tag_scores)
print(tag_to_ix)
print(training_data[0][0])

tensor([[-0.6032, -0.9824, -2.5444],
        [-2.5161, -0.0940, -4.7136],
        [-2.9137, -6.6433, -0.0572],
        [-0.1075, -3.0716, -2.8898],
        [-4.3054, -0.0144, -7.1087]], grad_fn=<LogSoftmaxBackward>)
{'DET': 0, 'NN': 1, 'V': 2}
['The', 'dog', 'ate', 'the', 'apple']


C:\Users\lijie4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


### Exercise: Augmenting the LSTM part-of-speech tagger with character-level features
In the example above, each word had an embedding, which served as the inputs to our sequence model.
Let's augment the word embeddings with a representation derived from the characters of the word.
We expect that this should help significantly, since character-level information like affixes have
a large bearing on part-of-speech.  For example, words with the affix *-ly* are almost always tagged as adverbs in English.

Do do this, let $c_w$ be the character-level representation of word $w$.  Let $x_w$ be the word embedding as before.
Then the input to our sequence model is the concatenation of $x_w$ and $c_w$.  So if $x_w$ has dimension 5, and $c_w$ dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the characters of a word, and let $c_w$ be the final hidden state of this LSTM.
Hints:
* There are going to be two LSTM's in your new model.  The original one that outputs POS tag scores, and the new one that outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.  The character embeddings will be the input to the character LSTM.

In [107]:
training_data = [
    ("The dog ate the apple".split(), ["DET", 'NN', 'V', 'DET', 'NN']),
    ("Everybody read that book".split(), ['NN', 'V', 'DET', 'NN'])
]


# def prepare_sequence():

NameError: name 'str2hex' is not defined

# 8. Advanced: Dynamic Toolkits, Dynamic Programming, and the BiLSTM-CRF
### Dyanmic versus Static Deep Learning Toolkits
### 动态和静态深度学习库

pytorch 是动态深度学习库
keras.tensorflow.theano是静态深度学习库
动态和静态的区别是：
1. 在静态库中，我们定义一个计算图，编译图，然后输入实例。
2. 在动态库中，我们为每一个实例定义一个计算图，它从不编译，而是动态执行。


Pytorch is a *dynamic* neural network kit.  Another example of a dynamic kit is [Dynet](https://github.com/clab/dynet) (I mention this because working with Pytorch and Dynet is similar.  If you see an example in Dynet, it will probably help you implement it in Pytorch). 
The opposite is the *static* tool kit, which includes Theano, Keras, TensorFlow, etc.
The core difference is the following:
* In a static toolkit, you define a computation graph once, compile it, and then stream instances to it.
* In a dynamic toolkit, you define a computation graph *for each instance*.  It is never compiled and is executed on-the-fly

Without a lot of experience, it is difficult to appreciate the difference.
One example is to suppose we want to build a deep constituent parser.
Suppose our model involves roughly the following steps:
* We build the tree bottom up
* Tag the root nodes (the words of the sentence)
* From there, use a neural network and the embeddings of the words to find combinations that form constituents.  Whenever you form a new constituent, use some sort of technique to get an embedding of the constituent.In this case, our network architecture will depend completely on the input sentence. In the sentence "The green cat scratched the wall", at some point in the model, we will want to combine the span $(i,j,r) = (1, 3, \text{NP})$ (that is, an NP constituent spans word 1 to word 3, in this case "The green cat").

However, another sentence might be "Somewhere, the big fat cat scratched the wall".  In this sentence, we will want to form the constituent $(2, 4, NP)$ at some point.
The constituents we will want to form will depend on the instance.  If we just compile the computation graph once, as in a static toolkit, it will be exceptionally difficult or impossible to program this logic.  In a dynamic toolkit though, there isn't just 1 pre-defined computation graph.  There can be a new computation graph for each instance, so this problem goes away.

Dynamic toolkits also have the advantage of being easier to debug and the code more closely resembling the host language (by that I mean that Pytorch and Dynet look more like actual Python code than Keras or Theano).

I mention this distinction here, because the exercise in this section is to implement a model which closely resembles structure perceptron, and I believe this model would be difficult to implement in a static toolkit.  I think that the advantage of dynamic toolkits for linguistic structure prediction cannot be overstated.

### Bi-LSTM Conditional Random Field Discussion

双向LSTM 条件随机场 for 命名实体识别。

LSTM 标注 对于词性标注是足够了。 但是像CRF这样的序列模型对于NER上的高性能非常重要。


For this section, we will see a full, complicated example of a Bi-LSTM Conditional Random Field for named-entity recognition.  The LSTM tagger above is typically sufficient for part-of-speech tagging, but a sequence model like the CRF is really essential for strong performance on NER.  Familiarity with CRF's is assumed.  Although this name sounds scary, all the model is is a CRF but where an LSTM provides the features.  This is an advanced model though, far more complicated than any earlier model in this tutorial.  If you want to skip it, that is fine.  To see if you're ready, see if you can:


* Write the recurrence for the viterbi variable at step i for tag k.
* Modify the above recurrence to compute the forward variables instead.
* Modify again the above recurrence to compute the forward variables in log-space (hint: log-sum-exp)

If you can do those three things, you should be able to understand the code below.
Recall that the CRF computes a conditional probability.  Let $y$ be a tag sequence and $x$ an input sequence of words.  Then we compute
$$ P(y|x) = \frac{\exp{(\text{Score}(x, y)})}{\sum_{y'} \exp{(\text{Score}(x, y')})} $$

Where the score is determined by defining some log potentials $\log \psi_i(x,y)$ such that
$$ \text{Score}(x,y) = \sum_i \log \psi_i(x,y) $$
To make the partition function tractable, the potentials must look only at local features.

In the Bi-LSTM CRF, we define two kinds of potentials: emission and transition.  The emission potential for the word at index $i$ comes from the hidden state of the Bi-LSTM at timestep $i$.  The transition scores are stored in a $|T|x|T|$ matrix $\textbf{P}$, where $T$ is the tag set.  In my implementation, $\textbf{P}_{j,k}$ is the score of transitioning to tag $j$ from tag $k$.  So:

$$ \text{Score}(x,y) = \sum_i \log \psi_\text{EMIT}(y_i \rightarrow x_i) + \log \psi_\text{TRANS}(y_{i-1} \rightarrow y_i) $$
$$ = \sum_i h_i[y_i] + \textbf{P}_{y_i, y_{i-1}} $$
where in this second expression, we think of the tags as being assigned unique non-negative indices.

If the above discussion was too brief, you can check out [this](http://www.cs.columbia.edu/%7Emcollins/crf.pdf) write up from Michael Collins on CRFs.

### The Forward Algorithm in Log-Space and the Log-Sum-Exp Trick

As hinted at above, computing the forward variables requires using a log-sum-exp.  I want to explain why, since it was a little confusing to me at first, and many resources just present the forward algorithm in potential space.  The recurrence for the forward variable at the $i$'th word for the tag $j$, $\alpha_i(j)$, is
$$ \alpha_i(j) = \sum_{j' \in T} \psi_\text{EMIT}(j \rightarrow i) \times \psi_\text{TRANS}(j' \rightarrow j) \times \alpha_{i-1}(j') $$

This is numerically unstable, and underflow is likely.  It is also inconvenient to work with proper non-negative potentials in our model.  We instead want to compute $\log \alpha_i(j)$.  What we need to do is to multiply the potentials, which corresponds to adding log potentials.  Then, we have to sum over tags, but what is the corresponding operation to summing over tags in log space?  It is not clear.  Instead, we need to transform out of log-space, take the product of potentials, do the sum over tags, and then transform back to log space.  This is broken down in the revised recurrence below:

$$ \log \alpha_i(j) = \log \overbrace{\sum_{j' \in T} \exp{(\log \psi_\text{EMIT}(j \rightarrow i) + \log \psi_\text{TRANS}(j' \rightarrow j) + \log \alpha_{i-1}(j'))}}^\text{transform out of log-space and compute forward variable} $$

If you carry out elementary exponential / logarithm identities in the stuff under the overbrace above, you will see that it computes the same thing as the first recurrence, then just takes the logarithm.  Log-sum-exp appears a fair bit in machine learning, and there is a [well-known trick](https://en.wikipedia.org/wiki/LogSumExp) to computing it in a numerically stable way.  I use this trick in my log_sum_exp function below (I don't think Pytorch provides this function in its library).

### Implementation Notes

The example below implements the forward algorithm in log space to compute the partition function, and the viterbi algorithm to decode.  Backpropagation will compute the gradients automatically for us.  We don't have to do anything by hand.

The implementation is not optimized.  If you understand what is going on, you'll probably quickly see that iterating over the next tag in the forward algorithm could probably be done in one big operation.  I wanted to code to be more readable.  If you want to make the relevant change, you could probably use this tagger for real tasks.

In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
torch.manual_seed(1)

In [3]:
torch.max?

In [5]:
a = torch.randn(1,3)
print(a)
torch.max(a, 1)

tensor([[ 0.6213, -0.4519, -0.1661]])


torch.return_types.max(values=tensor([0.6213]), indices=tensor([0]))

In [2]:
def to_scalar(var):
    return var.view(-1).data.tolist()[0]

def argmax(vec):
    #return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return to_scalar(idx)

## compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)] ## 获取每一个字的最大概率值
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))


In [ ]:
class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)
        
        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim/2, num_layers=1, bidirectional=True)
        ## Maps the output of the LSTM into tag space
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
        
        ## Matrix of transition parameters. Entry i,j is the score of transition to i from j
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))
        
        ## these two statements enforce the constraint that we never transfer
        ## to the start tag, and we never transfer from the stop tag(the model
        ## would probably learn this anyway, so this enforcement is likely unimportant)
        self.transitions.data[tag_to_ix[START_TAG],:] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000
        
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        return (autograd.Variable(torch.randn(2,1,self.hidden_dim)),
               autograd.Variable(torch.randn(2,1,self.hidden_dim)))
    
    def _forward_alg(self, feats):
        ## do the forward algorithm to compute the partition function
        init_alphas = torch.Tensor(1, self.tagset_size).fill_(-10000.)
        ## START_TAG has all of the score
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.
        
        ## Wrap in a variable so that we will get automatic backprop
        forward_var = autograd.Variable(init_alphas)
        # Iterate through the sentence
        for feat in feats:
            alphas_t = [] # the forward variables at this timestep
            for next_tag in range(self.tagset_size):
                ## broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to next_tag from i 
                trans_score = self.transitions[next_tag].view(1,-1)
                # the ith entry of next_tag_var is the value for the edge(i->next_tag)
                # before we do log_sum_exp
                next_tag_var = forward_var + trans_score + emit_score
                # the forward variable for this tag is log-sum-exp of all the scores
                alphas_t.append(log_sum_exp(next_tag_var))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

def _get_lstm_features(self, sentence):
    self.hidden = self.init_hidden()
    embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
    lstm_out, self.hidden = self.lstm(embeds)
    lstm_out = lstm_out.view(len(sentence), self.hidden_dim) ## view 如何理解
    lstm_feats = self.hidden2tag(lstm_out)
    return lstm_feats

def _score_sentence(self, feats, tags):
    # gives the score of a provided tag sequence
    score = autograd.Variable(torch.Tensor([0]))
    tags = torch.cat([torch.LongTensor([self.tag_to_ix[START_TAG]]), tags])
    for i, feats in enumerate(feats):
        score = score + self.transitions[tags[i+1], tags[i]] + feat[tags[i+1]]
    score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
    return score

def _viterbi_decode(self, feats):
    backpointers = []
    # initialize the viterbi variables in the log space
    init_vvars = torch.Tensor(1, self.tagset_size).fill_(-10000.)
    init_vvars[0][self.tag_to_ix[START_TAG]] = 0
    
    # forward_var at step i holds the viterbi variables for step i-1
    forward_var = autograd.Variable(init_vvars)
    for feat in feats:
        bptrs_t = [] # holds the backpointers for this step
        viterbivars_t = [] # holds the viterbi variables for this step
        
        for next_tag in range(self.tagset_size):
            # next_tag_var[i] holds the viterbi variable for tag i at the previous step.
            # plus the score of transitioning from tag i to next_tag
            # we don't include the emission scores here because the max does not depend on thme
            next_tag_var = forward_var + self.transitions[next_tag]
            best_tag_id = argmax(next_tag_var)
            
            
    


            

        
        
        